In [1]:
import torch


torch.cuda.is_available()

True

# Fine-Tuning

In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

[2024-01-20 15:04:45,421] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
# dataset = load_dataset("takaaki-inada/databricks-dolly-15k-ja-zundamon", split="train")
dataset = load_dataset("kunishou/databricks-dolly-15k-ja", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset.to_pandas()

,input,index,instruction,category,output
0,ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty ...,0,ヴァージン・オーストラリア航空はいつから運航を開始したのですか？,closed_qa,ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の...
1,,1,魚の種類はどっち？イコクエイラクブカとロープ,classification,イコクエイラクブカ
2,,2,ラクダはなぜ水なしで長く生きられるのか？,open_qa,ラクダは、長時間にわたってエネルギーと水分で満たされた状態を保つために、腰の脂肪を利用しています。
3,,3,アリスの両親には3人の娘がいる：エイミー、ジェシー、そして三女の名前は？,open_qa,三女の名前はアリス
4,小森田は1981年7月10日、熊本県に生まれる。高校卒業後、2000年にJ1リーグのアビスパ...,4,小森田友明はいつ生まれたの？,closed_qa,小森田友明は1981年7月10日に生まれました。
...,...,...,...,...,...
15010,,15010,変更を受け入れるにはどうしたらよいですか,brainstorming,変化を受け入れて、違いを見る
15011,レーザーは、電磁波の誘導放出に基づく光増幅の過程で光を放出する装置である。レーザーという言葉...,15011,レーザーとは何か、誰が作ったのか？,summarization,レーザーは、電磁波源から光を放出する装置である。 最初のレーザーは、チャールズ・H・タウン...
15012,,15012,ロードバイクとマウンテンバイクの違いは何ですか？,open_qa,ロードバイクはアスファルトやセメントの上を走ることを想定して作られており、細いタイヤが装着さ...
15013,,15013,不動産投資業界において、GISはどのように役立っているのでしょうか。,general_qa,不動産投資家は、ビジネスを展開する市場や場所に関する競争力を高めるために、正確で精度の高いロ...


In [9]:
import pathlib

In [10]:
model_file = "../data/ELYZA-japanese-Llama-2-7b-fast-instruct-q8_0.gguf"
pathlib.Path(model_file).exists()

True

In [11]:
from llama_cpp import Llama

n_gqa = 8 if "70b" in model_file else 1
llm = Llama(model_path=model_file, n_gqa=n_gqa, n_gpu_layers=16)
llm

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 2 CUDA devices:
  Device 0: NVIDIA GeForce GTX 1080 Ti, compute capability 6.1, VMM: yes
  Device 1: NVIDIA GeForce GTX 1080 Ti, compute capability 6.1, VMM: yes
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ../data/ELYZA-japanese-Llama-2-7b-fast-instruct-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = ELYZA-japanese-Llama-2-7b-fast-instruct
llama_model_loader: - kv   2:      general.source.huggingface.repository str              = elyza/ELYZA-japanese-Llama-2-7b-fast-...
llama_model_loader: - kv   3:                   llama.tensor_data_layout str              = Meta 

In [13]:
llm.model

94634082358000

In [17]:
from peft import LoraConfig, get_peft_model
from peft.utils.peft_types import TaskType

In [ ]:
# peftの設定は読み込んだモデルをget_peft modelでパラメーターを凍結する際に指定します。
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

# creating model
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = llm
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# # output
# """
# Downloading (…)lve/main/config.json: 100%
# 747/747 [00:00<00:00, 59.3kB/s]
# Downloading pytorch_model.bin: 100%
# 14.1G/14.1G [00:39<00:00, 280MB/s]
# trainable params: 3,932,160 || all params: 7,072,948,224 || trainable%: 0.055594355783029126
# """

In [ ]:
# # peftの設定は読み込んだモデルをget_peft modelでパラメーターを凍結する際に指定します。
# peft_config = PrefixTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=30)
#
# # creating model
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()
#
# # output
# """
# Downloading (…)lve/main/config.json: 100%
# 715/715 [00:00<00:00, 52.6kB/s]
# Downloading model.safetensors: 100%
# 1.12G/1.12G [00:45<00:00, 28.4MB/s]
# trainable params: 1,474,560 || all params: 560,689,152 || trainable%: 0.26299064191632515
# """